<a href="https://colab.research.google.com/github/6uan/LaTeX-OCR-Testing/blob/main/notebooks/LaTeX_OCR_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a LaTeX OCR model
In this brief notebook I show how you can finetune/train an OCR model.

I've opted to mix in handwritten data into the regular pdf LaTeX images. For that I started out with the released pretrained model and continued training on the slightly larger corpus.

In [1]:
!pip install pix2tex[train] -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.9 MB/s eta 0:00:00


In [2]:
import os
!mkdir -p LaTeX-OCR
os.chdir('LaTeX-OCR')

In [3]:
!pip install gpustat -q
!pip install opencv-python-headless==4.8.1.78 -U -q
!pip install --upgrade --no-cache-dir gdown -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.3 MB/s eta 0:00:00


In [5]:
import os
import shutil
import random
from glob import glob

# Create the necessary directories if they don't exist
!mkdir -p dataset/data
!mkdir -p dataset/data/images  # Ensure this directory is correctly referenced

# Download the datasets using gdown
!gdown -O dataset/data/HANDWRTN.zip --id 18-bFQV5m1Ir1pq8deQiAkw743KpBSBN3         # NEW DATA
!gdown -O dataset/data/pdf.zip --id 176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ              # PDF IMAGES DATA
!gdown -O dataset/data/pdfmath.txt --id 1QUjX6PFWPa-HBWdcY-7bA5TRVUnbyS1D          # PDF MATH DATA

# Unzip the downloaded datasets
!unzip -q dataset/data/HANDWRTN.zip -d dataset/data
!unzip -q dataset/data/pdf.zip -d dataset/data

# Define the number of validation images you want
number_of_val_images = 1000

# Change directory to where the images are expected to be after unzipping
images_path = 'dataset/data/images'
val_images_path = 'dataset/data/valimages'

handwrtn_txt = '/content/LaTeX-OCR/dataset/data/HANDWRTN/HANDWRTN_math.txt'

# Check if the 'images' directory exists and has files
if os.path.exists(images_path) and os.path.isdir(images_path):
    # Get all the image files in the 'images' directory
    all_images = glob(os.path.join(images_path, '*'))

    # Shuffle the list of images
    random.shuffle(all_images)

    # Ensure the validation directory exists
    os.makedirs(val_images_path, exist_ok=True)

    # Move a subset of images to the 'valimages' directory
    for img in all_images[:number_of_val_images]:
        shutil.move(img, val_images_path)

    # The remaining files in 'images' are your training set
else:
    print(f"The directory {images_path} does not exist or is not a directory.")


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=18-bFQV5m1Ir1pq8deQiAkw743KpBSBN3
From (redirected): https://drive.google.com/uc?id=18-bFQV5m1Ir1pq8deQiAkw743KpBSBN3&confirm=t&uuid=b0470252-7722-4b5f-9d3f-922d90d20a99
To: /content/LaTeX-OCR/dataset/data/HANDWRTN.zip
100% 344M/344M [00:01<00:00, 199MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ
From (redirected): https://drive.google.com/uc?id=176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ&confirm=t&uuid=2bdc06a4-8fa2-41ac-a812-f84aec659606
To: /content/La

Now we generate the datasets. We can string multiple datasets together to get one large lookup table. The only thing saved in these pkl files are image sizes, image location and the ground truth latex code. That way we can serve batches of images with the same dimensionality.

Need Complete Path to make [15] and [16] work!

In [6]:
!pwd

/content/LaTeX-OCR


In [7]:
!python -m pix2tex.dataset.dataset -i dataset/data/images dataset/data/train -e /content/LaTeX-OCR/dataset/data/HANDWRTN_math.txt dataset/data/pdfmath.txt -o dataset/data/train.pkl


Generate dataset
0it [00:00, ?it/s]
100% 158480/158480 [00:02<00:00, 62678.98it/s]


In [8]:
!python -m pix2tex.dataset.dataset -i dataset/data/valimages dataset/data/val -e /content/LaTeX-OCR/dataset/data/HANDWRTN_math.txt dataset/data/pdfmath.txt -o dataset/data/val.pkl

Generate dataset
100% 1000/1000 [00:00<00:00, 66994.17it/s]
100% 6765/6765 [00:00<00:00, 58156.08it/s]


In [9]:
# download the weights we want to fine tune
!curl -L -o weights.pth https://github.com/lukas-blecher/LaTeX-OCR/releases/download/v0.0.1/weights.pth

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 97.3M  100 97.3M    0     0  93.8M      0  0:00:01  0:00:01 --:--:--  134M


In [10]:
# If using wandb
!pip install -q wandb
# you can cancel this if you don't wan't to use it or don't have a W&B acc.
#!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [11]:
# generate colab specific config (set 'debug' to true if wandb is not used)
!echo {backbone_layers: [2, 3, 7], betas: [0.9, 0.999], batchsize: 15, bos_token: 1, channels: 1, data: dataset/data/train.pkl, debug: true, decoder_args: {'attn_on_attn': true, 'cross_attend': true, 'ff_glu': true, 'rel_pos_bias': false, 'use_scalenorm': false}, dim: 256, encoder_depth: 4, eos_token: 2, epochs: 5, gamma: 0.9995, heads: 8, id: null, load_chkpt: 'weights.pth', lr: 0.001, lr_step: 30, max_height: 192, max_seq_len: 512, max_width: 672, min_height: 32, min_width: 32, model_path: checkpoints, name: mixed, num_layers: 4, num_tokens: 8000, optimizer: Adam, output_path: outputs, pad: true, pad_token: 0, patch_size: 16, sample_freq: 2000, save_freq: 1, scheduler: StepLR, seed: 42, temperature: 0.2, test_samples: 5, testbatchsize: 20, tokenizer: dataset/tokenizer.json, valbatches: 100, valdata: dataset/data/val.pkl} > colab.yaml

{backbone_layers: [2, 3, 7],
 betas: [0.9, 0.999],
 batchsize: 10,
 bos_token: 1,
 channels: 1,
 data: dataset/data/train.pkl,
 debug: true,
 decoder_args: {'attn_on_attn': true, 'cross_attend': true, 'ff_glu': true, 'rel_pos_bias': false, 'use_scalenorm': false},
 dim: 256,
 encoder_depth: 4,
 eos_token: 2,
 epochs: 50,
 gamma: 0.9995,
 heads: 8,
 id: null,
 load_chkpt: 'weights.pth',
 lr: 0.001,
 lr_step: 30,
 max_height: 192,
 max_seq_len: 512,
 max_width: 672,
 min_height: 32,
 min_width: 32,
 model_path: checkpoints,
 name: mixed,
 num_layers: 4,
 num_tokens: 8000,
 optimizer: Adam,
 output_path: outputs,
 pad: false,
 pad_token: 0,
 patch_size: 16,
 sample_freq: 2000,
 save_freq: 1,
 scheduler: StepLR,
 seed: 42,
 temperature: 0.2,
 test_samples: 5,
 testbatchsize: 20,
 tokenizer: dataset/tokenizer.json,
 valbatches: 100,
 valdata: dataset/data/val.pkl}

In [12]:
!python -m pix2tex.train --config colab.yaml

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /content/LaTeX-OCR/wandb/run-20231204_205605-ouump11v
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run mixed
wandb: ⭐️ View project at https://wandb.ai/6uannn/uncategorized
wandb: 🚀 View run at https://wandb.ai/6uannn/uncategorized/runs/ouump11v
Loss: 0.0391:  19% 1999/10460 [07:47<31:55,  4.42it/s]
  0% 0/398 [00:00<?, ?it/s]
BLEU: 0.848, ED: 1.03e-01, ACC: 0.632:   0% 0/3

In [13]:
!python eval.py -d path/to/crohme_test.pkl -c path/to/latest/checkpoint --config path/to/config.yaml!

python3: can't open file '/content/LaTeX-OCR/eval.py': [Errno 2] No such file or directory
